In [81]:
import pandas as pd
import numpy as np
import os
import warnings
import math

import statsmodels as sm
import statsmodels.stats.weightstats

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Prepare data

In [82]:
data = pd.read_csv('data/DS0001/04690-0001-Data.tsv',sep='\t')
data.shape

(3617, 4564)

In [83]:
data.head()

V1  V3  V4    V5  V6  V7  V8         V9   V21  V33  V90  V91  V92  V93  \
0   1   5  28  1986   1   1   1  390400411   381    1    2  -99    1  -99   
1   2   5  28  1986   1   1   2  341202411  2040    1    2  -99    1  -99   
2   3   5  21  1986   1   1   3  341206211  2047    1    2  -99    1  -99   
3   4   5  19  1986   1   1   4          0  5199   51    2  -99    1  -99   
4   5   5  18  1986   5   1   5  390602111  1306    1    2  -99    1  -99   

   V94  V95  V100  V101  V102  V103  V104  V105  V106  V107  V108  V109  V110  \
0  -99  -99     1     1     1     2    69   -99   -99   -99   -99   -99   -99   
1  -99  -99     2     4     1     1    44     2     2    42    11     1    15   
2  -99  -99     3     1     1     1    75   -99   -99   -99   -99   -99   -99   
3  -99  -99     0     2     1     1    25    21     1    45   -99   -99   -99   
4  -99  -99     5     2     1     2    30    11     1     7   -99   -99   -99   

   V111  V112  V113  V114  V115  V116  V117  V118  V119  V120  V121  V122  \
0   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
1    12     2    17   -99   -99   -99   -99   -99   -99   -99   -99   -99   
2   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
3   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
4   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   

   V123  V124  V125  V126  V127  V128  V129  V130  V131  V132  V133  V134  \
0   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
1   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
2   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
3   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   
4   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99   

   V135  V136  V137  V138  V201  V202  V203  V204  V205  V206  V207  V208  \
0   -99   -99   -99   -99     5     0   -99   -99   -99   -99   -99   -99   
1   -99   -99   -99     5     5     0   -99   -99   -99   -99   -99   -99   
2   -99   -99   -99   -99     1     4     1    55     1    37     1    31   
3   -99   -99   -99     1     5     0   -99   -99   -99   -99   -99   -99   
4   -99   -99   -99     5     5     0   -99   -99   -99   -99   -99   -99   

   V209  V210  V211  V212  V213  V214  V215  V216  V217  V218  V219  V220  \
0   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0     6   
1   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0     2   
2     2    45   -99   -99   -99   -99   -99   -99   -99   -99    11     5   
3   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0     3   
4   -99   -99   -99   -99   -99   -99   -99   -99   -99   -99     0     3   

   V221  V222  V223  V224  V225  V301  V302  V303  V304  V305  V306  V307  \
0     2     6     4     4     3     1    40   -99   -99   -99   -99    50   
1     2     3     2     2     2     2    19   -99   -99   -99   -99    96   
2     1     6     1     1     1     2    52   -99   -99   -99   -99    96   
3     2     6     2     3     2     2    52    58    40   -99   -99     8   
4     1     6     2     2     3     2     3    29   -99   -99   -99    33   

   V308  V309  V310  V311  V312  V313  V314  V315  V316  V317  V318  V319  \
0    40   -99   -99   -99     3     2     2     2   -96   -96   -96     2   
1   -99   -99   -99   -99     2     1     4     2     1     2     1     2   
2   -99   -99   -99   -99     2     2     2     3     2     2     2     1   
3   -99   -99   -99   -99     1     1     2     2     1     3     2     4   
4    29    25   -99   -99     1     1     2     2     3     4     3     1   

   V320  V321  V322  V323  V324  V325  V326  V327  V328  V329  V330  V331  \
0     2     3     1     4     4     1     3     2   -96   -96   -96   -96   
1     3     6     1     2     1     2   -99     1     1     4     4     1   
2     3     2     2     1     1     1     4 

In [208]:
feature_map = {"V2102": "Race", "V103": "Sex", "V2000": "Age", "V2007": "Education", "V2020": "Income", "V2637": "Smoking", "V2623": "BMI", "V2681": "Chronic medical conditions", "V13214": "Physical Activity Index", "V2203": "Depressive symptomatology", "V915": "General health", "V1860": "Weight", "V15003": "Response pattern", "V836": "Stroke wave 1", "V4838": "Stroke wave 2", "V10225": "Stroke wave 3", "V12305": "Stroke wave 4", "V15944": "Stroke wave 5", "V12302": "Any stroke"}

In [193]:
# select relevant columns
data_select_cols = data[feature_map.keys()]
data_select_cols.rename(columns=feature_map, inplace=True)
data_select_cols.shape


(3617, 19)

In [212]:
def get_recent_stroke_if_died(x):
    response_pattern = str(x["Response pattern"])
    if "4" not in response_pattern:
        return 0 # patient did not die
    death_wave = response_pattern.index("4")
    if x[f"Stroke wave {death_wave}"] == 1:
        return 1
    else:
        return 0

def get_any_stroke_if_died(x):
    response_pattern = str(x["Response pattern"])
    if "4" not in response_pattern:
        return 0 # patient did not die
    for i in range(5):
        if x[f"Stroke wave {i + 1}"] == 1:
            return 1
    return 0

def get_ever_had_stroke(x):
    response_pattern = str(x["Response pattern"])
    if "4" not in response_pattern:
        return 0 # patient did not die
    elif x["Any stroke"] == 1:
        return 1
    else:
        return 0


In [215]:
# construct additional columns and translate data
data_sample = data_select_cols.copy()
data_sample = data_sample[(data_sample["Race"] == 1) | (data_sample["Race"] == 2)] # 1 = white, 2 = Black
data_sample["Educational attainment"] = data_sample.apply(lambda x: 1 if x["Education"] >= 12 else 0, axis=1)
data_sample["Obese"] = data_sample.apply(lambda x: 1 if x["BMI"] > 30 else 0, axis=1)
data_sample["General health binary"] = data_sample.apply(lambda x: 1 if x["General health"] in [1, 2, 3] else 0, axis=1)
data_sample["Dead"] = data_sample.apply(lambda x: get_any_stroke_if_died(x), axis=1)
data_sample.drop(columns=['Stroke wave 1', 'Stroke wave 2', 'Stroke wave 3', 'Stroke wave 4', 'Stroke wave 5','Response pattern', 'Any stroke'], inplace=True)

In [216]:
data_sample["Dead"].sum()

107

## Exploring weighted statistics
On page 4, in paragraph 1 of "Statistical Analysis," they note that "Due to the complex survey design (due to the multistage sampling that involved clustering and stratification) of the ACL, Taylor series linearization was used to estimate the standard errors using the sampling weights (due to stratification, clustering, and non-response). **All proportions have applied weights**. Mean and frequency tables were used to describe our sample.

Took me a while, but I eventually found the *final* weights variable (I think!):
https://www.icpsr.umich.edu/web/ICPSR/studies/04690/datasets/0001/variables/V5859?archive=icpsr

In [136]:
data_sample.loc[data_sample['Weight'] != -99.0]['Weight'].describe()

count    3361.000000
mean        0.968658
std         0.850849
min         0.060000
25%         0.350000
50%         0.660000
75%         1.350000
max         9.620000
Name: Weight, dtype: float64

In [137]:
# Hypothesis: "inapplicable weights" are given values of -99.0, 
# but DescrStatsW does not treat those properly. Thus, weight those 
# examples to get the results from the paper...

# Impute this magic sauce weight value for missing numbers. *shrug*
#data_sample.loc[data_sample['Weight'] == -99.0, 'Weight'] = 0
#data_sample = data_sample[data_sample['Weight'] != -99.0]

# I thought we might want to impute means to get the results, but that ended
# up overweight things
# NOTE: I checked, no weights were 0.0, so we are safe to do this
# inapp_size = len(data_sample.loc[data_sample['Weight'] == -99.0])
# mean_weight = sum(data_sample['Weight'])/(len(data_sample['Weight'])-inapp_size)

## 3.1 Descriptive Statistics

In [138]:
print(f"Total: {data_sample.shape[0]}")
print(f"Total number Black participants: {data_sample.loc[data_sample['Race'] == 2].shape[0]}")
print(f"Total number white participants: {data_sample.loc[data_sample['Race'] == 1].shape[0]}")

Total: 3361
Total number Black participants: 1156
Total number white participants: 2205


In [218]:
white_sample = data_sample[data_sample["Race"] == 1]
black_sample = data_sample[data_sample["Race"] == 2]
mean_cols = ['Age', 'Men percent', 'Women percent', 'Educational attainment', 'Education', 'Income', 'General health', 'Chronic medical conditions', 'General health binary', 'Smoking', 'Obese']

In [140]:
def get_adjusted_means(data_sample):
    means = np.around(sm.stats.weightstats.DescrStatsW(data_sample, weights=data_sample['Weight']).mean, 4)
    #CIs = sm.stats.weightstats.DescrStatsW(data_sample, weights=data_sample['Weight']).tconfint_mean(alpha=0.05)
    #CIs = zip(np.around(CIs[0],3),np.around(CIs[1],3)) # upper, lower tuples
    #return pd.DataFrame(data=[means, CIs], columns=data_sample.columns)
    result = pd.DataFrame(data=[means], columns=data_sample.columns)
    result['Women percent'] = result.apply(lambda x: (x['Sex'] - 1) * 100, axis=1)
    result['Men percent'] = result.apply(lambda x: 100 - x['Women percent'], axis=1)
    result['Smoking'] *= 100
    result['General health binary']*= 100
    result['Obese']*=100
    return result[mean_cols]


In [141]:
get_adjusted_means(data_sample)


,Age,Men percent,Women percent,Educational attainment,Education,Income,General health,Chronic medical conditions,General health binary,Smoking,Obese
0,47.7896,47.26,52.74,0.7597,12.5301,5.4058,2.3007,0.7951,85.06,30.41,14.46


In [142]:
get_adjusted_means(white_sample)


,Age,Men percent,Women percent,Educational attainment,Education,Income,General health,Chronic medical conditions,General health binary,Smoking,Obese
0,47.9828,47.82,52.18,0.782,12.6891,5.5634,2.2829,0.7781,85.97,29.65,13.52


In [143]:
get_adjusted_means(black_sample)

,Age,Men percent,Women percent,Educational attainment,Education,Income,General health,Chronic medical conditions,General health binary,Smoking,Obese
0,46.3685,43.18,56.82,0.5953,11.3603,4.2465,2.4321,0.9199,78.38,35.95,21.44


## 3.2 Cerebrovascular Deaths

In [219]:
print(f'Num died: {data_sample["Dead"].sum()}')
print(f'White died: {white_sample["Dead"].sum()}({(white_sample["Dead"].sum()/data_sample["Dead"].sum())*100}%)')
print(f'Black died: {black_sample["Dead"].sum()}({(black_sample["Dead"].sum()/data_sample["Dead"].sum())*100}%)')

Num died: 107
White died: 69(64.48598130841121%)
Black died: 38(35.51401869158878%)


In [220]:
print(f'Num died: {data_sample["Dead"].sum()}')
num_dead_and_obese = data_sample.loc[(data_sample["Dead"] == 1) & (data_sample["Obese"] == 1)].shape[0]
print(f'Obese died: {num_dead_and_obese}({(num_dead_and_obese/data_sample["Dead"].sum())*100}%)')
print(f'Not obese died: {data_sample["Dead"].sum() - num_dead_and_obese}({((data_sample["Dead"].sum() - num_dead_and_obese)/data_sample["Dead"].sum())*100}%)')

Num died: 107
Obese died: 28(26.168224299065418%)
Not obese died: 79(73.83177570093457%)
